In [33]:
import os
from pathlib import Path

path = Path('model.pkl')

!pip install -Uqq fastai

In [34]:
print(f"File exists: {path.exists()}")
print(f"File size: {path.stat().st_size if path.exists() else 'N/A'} bytes")
print(f"Full path: {path.absolute()}")
import zipfile

try:
    with zipfile.ZipFile(path, 'r') as zip_file:
        print("File is a valid ZIP archive")
except zipfile.BadZipFile:
    print("File is corrupted or not a valid ZIP archive")
except FileNotFoundError:
    print("File not found")


File exists: True
File size: 94592082 bytes
Full path: /Users/kalanjarvis-loewen/Desktop/Coding/learning/ml/LeanAI/core/model.pkl
File is a valid ZIP archive


In [35]:
from fastai.vision.all import *
model = load_learner("model.pkl")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [36]:
def preprocess_img(pre_img):
    img = Image.open('images/0_image_1.jpg').convert('RGB')
    return transform(img).unsqueeze(0)

In [37]:
def predict_image(img):

    if img is None:
        return "Please upload an image first"
    
    try:
        print(f"Input type: {type(img)}")

        pil_img = PILImage.create(img)

        
        bf, _, preds = model.predict(pil_img)
        
        return float(preds[0])
        
    except Exception as e:
        return print(f"error: {e}")

pred = predict_image('images/0_image_1.jpg')               
print(pred)

Input type: <class 'str'>


7.862442493438721


In [38]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Body Fat Prediction")
    
    with gr.Row():
        image_input = gr.Image(label="Upload Image")
        output = gr.Number(label="Predicted Value")
    
    predict_btn = gr.Button("Predict")
    predict_btn.click(predict_image, inputs=image_input, outputs=output)

#demo.launch(share=True)



In [41]:
from fastapi import FastAPI, Request
import nest_asyncio
import uvicorn
import threading

app = FastAPI()

@app.post("/model/predict")
async def predict(request: Request):
    data = await request.json()
    # Do your model stuff here
    return {"received": data}

# Patch event loop so uvicorn can run inside Jupyter
nest_asyncio.apply()

# Run uvicorn in a background thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=7866)

threading.Thread(target=run).start()

INFO:     Started server process [95460]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7866 (Press CTRL+C to quit)
